In [20]:
!pip install scrapy-fake-useragent


### playnommore
http://playnomore.co.kr

In [16]:
import scrapy
import requests

from scrapy.http import TextResponse
from fake_useragent import UserAgent

In [17]:

!scrapy startproject playnomore

Error: scrapy.cfg already exists in /Users/jhkim/Documents/dev/TIL_/playnomore


In [18]:
%%writefile playnomore/playnomore/items.py
class PlaynomoreItem(scrapy.Item):
    title = scrapy.Field()
    price = scrapy.Field()
    img = scrapy.Field()
    link = scrapy.Field()

Overwriting playnomore/playnomore/items.py


In [19]:
url = "http://www.playnomore.co.kr/category/bag/24/"
#headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}
headers = {"User-Agent": UserAgent().chrome}
req = requests.get(url, headers = headers)
reponse = TextResponse(req.url, body=req.text, encoding = "utf-8")
response


NameError: name 'response' is not defined

In [ ]:
links = response.xpath('//*[@id="contents"]/div[2]/ul/li/div[1]/a/@href').extreat()
links = map(lambda data: reponse.urljoin(data), links)


In [21]:
//*[@id="contents"]/div[1]/div[1]/div[1]/div[2]/ul/li[1]/img

SyntaxError: invalid syntax (<ipython-input-21-3a2d4eecca95>, line 1)

# spider 

In [ ]:
%%writefile playnomore/playnomore/spiders/spiders.py
import scrapy
from playnomore.items import PlaynomoreItem

class PlaynomoreSpider(scrapy.Spider):
    name = "Playnomore"
    # user-agent setting
    custom_settings = {
        "DOWNLOADER_MIDDLEWARES": {
            "scrapy.downloadmiddlewares.useragent.UserAgentMiddleware": None,
            "scrapy_fake_useragent.middlewares.RandomUserAgentMiddleware": 400,
        }
    }
    
    def start_requests(self):
        urls = ["http://www.playnomore.co.kr/category/bag/24/"] #start urls 에 넣은것과 같은데이터
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)
            
    def parse(self, response)
        links = response.xpath('//*[@id="contents"]/div[2]/ul/li/div[1]/a/@href').extreat()
        links = map(lambda data: reponse.urljoin(data), links)
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
    
    def parse_content(self, response):
        item = PlaynomoreItem()
        title1= response.xpath('//*[@id="contents"]/div[1]/div[1]/div[1]/div[2]/ul/li[1]/img')

# arg setting

# run scrpay 

In [ ]:
%%writefile run.sh
cd playnomore
scrapy crawl Playnomore -o playnomore.py

In [ ]:
!soruce run.sh

In [ ]:
pd.read_csv("playnomore/playnomore.sh")